In [48]:
!pip install neo4j

Код для выгрузки фильмов и людей

In [6]:
from kinopoisk_unofficial.kinopoisk_api_client import KinopoiskApiClient
from kinopoisk_unofficial.request.staff.staff_request import StaffRequest
from kinopoisk_unofficial.request.films.film_request import FilmRequest
from requests.exceptions import HTTPError
from typing import List


films = dict()


def parse_film_staff(film_id: int = None, token: str = "07a871a4-5e2b-48c4-8348-8428d54ab3bf") -> List:
    api_client = KinopoiskApiClient(token)
    request = StaffRequest(film_id)
    response = []
    try:
        response = api_client.staff.send_staff_request(request)
    except HTTPError as e:
        if e.response.status_code == 404:
            print(f"Staff ID not found.")
    except Exception as e:
        print(f"An error occurred: {e}")
    return response.items



def parse_film(film_id: int = None, token: str = "07a871a4-5e2b-48c4-8348-8428d54ab3bf"):
    api_client = KinopoiskApiClient(token)
    request = FilmRequest(film_id)
    response = None
    try:
        response = api_client.films.send_film_request(request)
    except Exception as e:
        print(f"Film with ID {film_id} does not exist or an error occurred")
    if response is not None:
        film = response.film
        return {
            'name_ru': film.name_ru,
            'name_original': film.name_original,
            'year': film.year,
            'genres': [genre.genre for genre in film.genres],
            'description': film.description
        }
    else:
        return None
    

token = "07a871a4-5e2b-48c4-8348-8428d54ab3bf"
for film_id in range(299,316):
    films[film_id] = parse_film(film_id,token) 

Film with ID 315 does not exist or an error occurred


Сохранение выгруженных данных

In [7]:
import csv

# Открываем файл для записи
with open('film_all_data_part_testing.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Записываем заголовок
    writer.writerow(["Film ID", "Film Name (RU)", "Film Name (Original)", "Film Year", "Film Genres", "Film Description", "Staff Info"])

    # Записываем данные для каждого фильма
    for film_id, film_data in films.items():
        if film_data is not None:
            film_name = film_data['name_ru']
            film_name_original = film_data['name_original']
            film_year = film_data['year']
            film_genres = ', '.join(film_data['genres'])
            film_description = film_data['description']    
            if film_description is not None:
                film_description = film_description.replace('\n', ' ').replace('\r', '')  # Удаляем символы новой строки и возврата каретки
            staff_info = parse_film_staff(film_id, token)
            staff_info_str = [f"{staff.name_ru} ({staff.staff_id}, {staff.profession_text}, {staff.description})" for staff in staff_info]
            writer.writerow([film_id, film_name, film_name_original, film_year, film_genres, film_description, staff_info_str])


Код для объединения нескольких выгрузок

In [ ]:
import csv

# Open the CSV files and create a DictReader object for each
with open('film_all_data_part1.csv', 'r', newline='', encoding='utf-8') as file1, \
     open('film_all_data_part2.csv', 'r', newline='', encoding='utf-8') as file2:
    reader1 = csv.reader(file1)
    reader2 = csv.reader(file2)

    # Open the output CSV file and create a DictWriter object
    with open('film_all_data_combined.csv', 'w', newline='', encoding='utf-8') as file_out:
        writer = csv.writer(file_out)

        # Write rows from the first CSV file
        for row in reader1:
            writer.writerow(row)

        # Skip the header of the second CSV file
        next(reader2)

        # Write rows from the second CSV file
        for row in reader2:
            writer.writerow(row)

Загрузка данных в NEO4j

In [8]:
from neo4j import GraphDatabase
import re
import pandas as pd


class gdbs: # Конструктор класса, принимающий параметры подключения к базе данных

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        self.data = None


    def read_data(self,path):  # Метод для чтения данных из CSV файла
        self.data = pd.read_csv(r"C:\Users\7ivan\PycharmProjects\Kinopoisk\film_all_data_combined.csv",encoding = "utf-8").values


    def close(self):  # Метод для закрытия подключения к базе данных
        self.driver.close()

    # Метод для добавления записи в базу данных
    def add_record(self, record):   # Здесь происходит обработка данных и формирование Cypher запроса для добавления записи в базу данных
        print('_'*100)
        film_id = record[0]
        film_dict = {"film_name": re.sub("[^a-zA-Z,а-яА-Я:.-]","",record[2]),
                     "film_name_ru": re.sub("[^a-zA-Z,а-яА-Я:.-]","",record[1]),
                     "film_url_id": record[0],
                     "film_year": record[3],
                     "film_genres": record[4] if "," not in record[4] else record[4].split(", "),
                     # "film_description": record[5],
                     }
        staff = [re.sub("'", '', i) for i in re.sub(r'"',"'",record[-1][2:-2]).split("',")]
        roles = staff.copy()
        sep = "("
        sep1 = ","
        sep2 = ')'
        roles = [f'({self.name_clear(x.split(sep)[0].strip())})-[:{self.name_clear(x.split(sep)[1].split(",")[1].strip())} {self.dict_clear(dict(roles = f"{x.split(sep)[1].split(sep1)[2].strip()}".split(sep2)), clear=False) if "None" not in x.split(sep)[1].split(sep1)[2] else ""}]->({film_dict["film_name"]})' for x in roles]
        staff = list(set([f'CREATE ({self.name_clear(x.split("(")[0].strip())}:Person {self.dict_clear(dict(name = x.split("(")[0].strip()))})' for x in staff]))
        staff = " ".join(staff)
        roles = "CREATE " + ", ".join(roles)

#
        string_dict= '{' + ",".join([f'{key}:"{val}"' for key, val in film_dict.items()]) + '}'
        str_ = f"CREATE ({film_dict['film_name']}:Movie {string_dict}) "
#
        complete = ' '.join([str_, staff, roles])
        with self.driver.session() as session:
            # r = session.execute_write(self.add_movie, film_dict)
            # r = session.execute_write(self.add_, staff)
            # r = session.execute_write(self.add_, roles)
            r = session.execute_write(self.add_, complete)

    # Метод для вывода всех связей в базе данных
    def print_all_relationships(self):
        with self.driver.session() as session: # Здесь выполняется Cypher запрос, который возвращает все узлы в базе данных
            result = session.run("MATCH (n) RETURN n")
            for record in result:
                print(record)


    def name_clear(self, name_str):  # Методы для обработки и очистки данных
        if len(name_str.split(' ')) == 1:
            return name_str
        else:
            return '_'.join([re.sub(r'\W', '' ,name_part) for i, name_part in enumerate(name_str.split(' '))])


    def writetxt(self, str_, name):
        file = open("{name}.txt", "w")
        file.write(str_)
        file.close()

    @staticmethod
    def add_movie(tx, data):
        string_dict= '{' + ",".join([f'{key}:"{val}"' for key, val in data.items()]) + '}'
        str_ = f"CREATE ({data['film_name']}:Movie {string_dict}) "
        tx.run(str_)


    def dict_clear(self, dict_, clear=True):
        if clear:
              for key, val in dict_.items():
                  key = key.replace(" ", '')
                  # val = val.replace(" ", '')
                  dict_[key] = val
              return '{' + ",".join([f'{key}:"{val}"' for key, val in dict_.items()]) + '}'
        else:
              return '{' + ",".join([f"{key}:{val}" for key, val in dict_.items()]) + '}'




    @staticmethod
    def add_(tx, data):
        tx.run(data)


    def clear_db(self,): # Метод для очистки базы данных
        with self.driver.session() as session:
            result = session.run("MATCH (n) DETACH DELETE (n)") # Здесь выполняется Cypher запрос, который удаляет все узлы в базе данных


    def csv_to_neo(self): # Метод для преобразования данных из CSV файла в формат, подходящий для Neo4j
        # for i in self.data[:20]:
        for i in self.data:
          try:
              self.add_record(i)
          except Exception as e:
              print(e)

# Создание экземпляра класса и выполнение операций с базой данных
gdb = gdbs("bolt://localhost:7687", "neo4j", "11111111")
gdb.read_data(r"C:\Users\7ivan\PycharmProjects\Kinopoisk\film_all_data_combined.csv")
gdb.clear_db()
# gdb.add_record(gdb.data[0])
gdb.csv_to_neo()
gdb.close()

____________________________________________________________________________________________________
____________________________________________________________________________________________________
____________________________________________________________________________________________________
____________________________________________________________________________________________________
____________________________________________________________________________________________________
____________________________________________________________________________________________________
____________________________________________________________________________________________________
____________________________________________________________________________________________________
____________________________________________________________________________________________________
___________________________________________________________________________________________

In [94]:
gdb.print_all_relationships()

<ipython-input-93-c38d7dac8e0f>:54: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with self.driver.session() as session:


<Record n=<Node element_id='4:f8dd5902-6679-495c-9497-725d6a35cefd:0' labels=frozenset({'Movie'}) properties={'film_genres': "['фантастика', 'боевик']", 'film_name': 'TheMatrixReloaded', 'film_name_ru': 'Матрица:Перезагрузка', 'film_url_id': '299', 'film_year': '2003'}>>
<Record n=<Node element_id='4:f8dd5902-6679-495c-9497-725d6a35cefd:1' labels=frozenset({'Person'}) properties={'name': 'Рэндолл Дак Ким'}>>
<Record n=<Node element_id='4:f8dd5902-6679-495c-9497-725d6a35cefd:2' labels=frozenset({'Person'}) properties={'name': 'Йен Блисс'}>>
<Record n=<Node element_id='4:f8dd5902-6679-495c-9497-725d6a35cefd:3' labels=frozenset({'Person'}) properties={'name': 'Фрэнки Стивенс'}>>
<Record n=<Node element_id='4:f8dd5902-6679-495c-9497-725d6a35cefd:4' labels=frozenset({'Person'}) properties={'name': 'Ярослава Турылёва'}>>
<Record n=<Node element_id='4:f8dd5902-6679-495c-9497-725d6a35cefd:5' labels=frozenset({'Person'}) properties={'name': 'Оуэн Патерсон'}>>
<Record n=<Node element_id='4:f8dd5

SyntaxError: invalid syntax (1612945206.py, line 1)